In [ ]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine


In [ ]:
df_url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-10-08/ipf_lifts.csv"
df = pd.read_csv(df_url)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.value_counts()

In [ ]:
df.index

In [ ]:
df.fillna(0).isna().sum()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
duplicates = df[df.duplicated(['name'], keep=False)]
uniq = df['name'].nunique()
print(duplicates)
print(uniq)

Мы видим данные о спортсменах с соревнованиях.
Дополнительные столбцы показывают информацию о соревновании, оборудовании, месте тренировки и весе и т. д.

2. Найдите рекорды каждого пола и каждого подразделения в каждом из 3
упражнений (жим лежа, приседания и становая тяга).

Тк в колонках, максимальное значение которых мне нужно вычислить есть значения NaN - функция max будет возвращать NaN. 
Необходимо строки с этими значениями игнорировать с помощью - .max(skipna=True)

In [ ]:
var_m = df[df['sex'] == 'M']
best3squat_kg_m = var_m['best3squat_kg'].max(skipna=True)
print(best3squat_kg_m)

best3bench_kg_m = var_m['best3bench_kg'].max(skipna=True)
print(best3bench_kg_m)

best3deadlift_kg_m = var_m['best3deadlift_kg'].max(skipna=True)
print(best3deadlift_kg_m)


In [ ]:
var_f = df[df['sex'] == 'F']
best3squat_kg_f = var_f['best3squat_kg'].max(skipna=True)
print(best3squat_kg_f)

best3bench_kg_f = var_f['best3bench_kg'].max(skipna=True)
print(best3bench_kg_f)

best3deadlift_kg_f = var_f['best3deadlift_kg'].max(skipna=True)
print(best3deadlift_kg_f)


3. Посчитайте количество побед каждого участника, принимая за победу 1-е
место. Сохраните результаты во вторичном DataFrame.

In [ ]:
winners = df[df['place'] == '1'].groupby('name')['place'].count().reset_index(name='wins')
df_winners = df.merge(winners, on='name', how='left').fillna({'wins': 0})
df_winners.head(100)
df_winners.to_csv('df_winners.csv', index=False)

In [ ]:
winners = df[df['place'] == '1'].groupby('name')['place'].count().reset_index(name='count_wins')
df_winners = df.merge(winners, on='name', how='left')
df_winners.to_csv('df_winners.csv', index=False)
print(df_winners)

4. Используйте DataFrame из упражнения 3, чтобы показать для каждой
комбинации подразделения и пола, участника с наибольшим количеством побед
среди участников, когда-либо участвовавших в этом дивизионе.

In [ ]:
max_wins_F = df_winners[df_winners['sex'] == 'F'].groupby('division').apply(lambda x: x.loc[x['count_wins'].idxmax()])
print(df_max_wins_F)

In [ ]:
max_wins_M = df_winners[df_winners['sex'] == 'M'].groupby('division').apply(lambda x: x.loc[x['count_wins'].idxmax()])
print(max_wins_M)

In [ ]:
max_wins = df_winners.groupby(['division', 'sex']).apply(lambda x: x.loc[x['count_wins'].idxmax()])
print(max_wins)

5. Подключитесь к базе данных employees и выполните с помощью pandas
последний запрос из финального проекта по SQL. Убедитесь, что результаты
загружены в pandas DataFrame.


/* 6. Написать запрос, который выводит строки с дубляжами. */

/* 
because student_no UNIQUE value, I used another columns for identifing duplicates
--
Adam Lee	adam.lee@mail.com	2000-01-01	2
Brandon Chen	brandon.chen@mail.com	2000-02-02	2
Cathy Wang	cathy.wang@mail.com	2001-03-03	2
*/

SELECT student_name, email, birth_date, COUNT(*)
FROM students
GROUP BY student_name, email, birth_date
HAVING COUNT(*) > 1
;


In [ ]:
user = '****'
password = '*********'
host = 'localhost'
port = '3306'
database = 'courses'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')



In [ ]:
my_sql_query = '''
SELECT student_name, email, birth_date, COUNT(*)
FROM students
GROUP BY student_name, email, birth_date
HAVING COUNT(*) > 1
'''

In [ ]:
df_courses = pd.read_sql_query(my_sql_query, con=engine)
print(df_courses)

   student_name                  email  birth_date  COUNT(*)
0      Adam Lee      adam.lee@mail.com  2000-01-01         2
1  Brandon Chen  brandon.chen@mail.com  2000-02-02         2
2    Cathy Wang    cathy.wang@mail.com  2001-03-03         2